# Travel Planner

A simple implementation with routing

In [68]:
import os
from dotenv import load_dotenv
from typing import Any, Dict, List, Optional, Literal, Set, TypedDict
from pydantic import BaseModel, Field, ValidationError
from IPython.display import display, Image
import json
# LangGraph StateGraph import (assume installed)
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

# LangChain imports (LLM & Chains)
from langchain_openai import ChatOpenAI
from langchain import LLMChain, PromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Tavily search
from langchain_tavily import TavilySearch

In [3]:
load_dotenv()

True

In [4]:
# Setup LLM clients using env keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment. Set it in .env or env vars before running.")

llm = ChatOpenAI(model="gpt-4", temperature=0.0)

In [5]:
# Setup Tavily client (assume TAVILY_API_KEY present)
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise RuntimeError("TAVILY_API_KEY not set in environment. Set it in .env or env vars before running.")

# instantiate tool
tavily_tool = TavilySearch(max_results = 5)

In [43]:
tavily_tool.invoke({"query": "Search Hotels in Mumbai from 2nd November 2025 for 3 nights, and return hotel name, rating, price, and amenities."})

{'query': 'Search Hotels in Mumbai from 2nd November 2025 for 3 nights, and return hotel name, rating, price, and amenities.',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.tripadvisor.com/Hotels-g304554-Mumbai_Maharashtra-Hotels.html',
   'title': 'THE 10 BEST Hotels in Mumbai, India 2025 (from $13) - Tripadvisor',
   'content': 'Find the right hotel for you · 1. Ginger Mumbai Airport · 2. Niranta Airport Transit Hotel & Lounge · 3. The Lalit Mumbai · 4. Taj Lands End,',
   'score': 0.7533401,
   'raw_content': None},
  {'url': 'https://www.expedia.com/Mumbai.d6050062.Destination-Travel-Guides',
   'title': 'Mumbai Vacation Packages 2025/2026 from $804 | Expedia',
   'content': 'The Latest Mumbai vacation packages · Fabhotel 7 Square · The Gordon House Hotel · Fabexpress Gulshan Grand · Hotel Marine Plaza Mumbai · West End Hotel.',
   'score': 0.6914979,
   'raw_content': None},
  {'url': 'https://www.orbitz.com/Mumbai-Hotels.d6050062.

In [79]:
class Constraints(BaseModel):
    origin: Optional[str] = None
    city: Optional[str] = None
    date: Optional[str] = None
    return_date: Optional[str] = None
    days: Optional[int] = None
    budget_per_person: Optional[float] = None
    interests: Optional[str] = None
    transport_constraints: Optional[Dict[str, Any]] = None
    hotel_constraints: Optional[Dict[str, Any]] = None

class TransportOption(BaseModel):
    provider: str
    mode: str
    depart: str
    arrive: str
    duration_min: Optional[str]
    price_per_person: Optional[str]
    notes: Optional[str] = None

class HotelOption(BaseModel):
    name: str
    rating: Optional[float]
    price_per_night: float
    distance_from_center_km: Optional[float]
    amenities: List[str] = Field(default_factory=list)
    notes: Optional[str] = None

class ItineraryItem(BaseModel):
    day: int
    morning: str
    afternoon: str
    evening: str
    notes: Optional[str] = None

class Intention(BaseModel):
    type: Literal["travel_plan","hotel_only","transport_only","unrelated"]

class PlannerState(TypedDict):
    goal: str
    intent: Optional[str]
    constraints: Optional[Constraints] = None
    transport_options: List[TransportOption] = Field(default_factory=list)
    hotel_options: List[HotelOption] = Field(default_factory=list)
    itinerary: List[ItineraryItem] = Field(default_factory=list)
    # subtasks: List[Subtask] = Field(default_factory=list)
    # policy: Dict[str, Any] = Field(default_factory=lambda: {"max_retries": 2, "worker_timeout": 12.0})
    final_plan: Optional[str] = None

In [7]:
# Supervisor LLM with intentation output
supervisor_llm = llm.with_structured_output(Intention, method="function_calling")

# Bind Tavily tool to LLM for worker nodes
worker_llm = llm.bind_tools([tavily_tool], tool_choice="auto")

In [8]:
# Supervisor prompt to classify user intent
supervisor_prompt = PromptTemplate(
    input_variables=["goal"],
    template=(
        "You are an expert supervisor of a Travel Planner"
        "Classify the user's travel planning intent based on the goal below.\n"
        "Possible intents are:\n"
        "1. travel_plan - planning a full trip\n"
        "2. hotel_only - only looking for hotel options\n"
        "3. transport_only - only looking for transport options\n"
        "4. unrelated - not related to travel planning or transport booking or hotel booking\n\n"
        "Goal: {goal}\n"
        "Respond with the intent type only."
    )
)

# Supervisor chain
supervisor_chain = supervisor_prompt | supervisor_llm

# Planner promt and chain
constraints_parser = PydanticOutputParser(pydantic_object=Constraints)
format_instructions = constraints_parser.get_format_instructions()

planner_prompt_template = f"""
You are an assistant that extracts travel constraints from a user query.
Return ONLY valid JSON that matches the Constraints model below.

Constraints model:
  "origin": string or null,
  "city": string or null,
  "date": string (YYYY-MM-DD) or null,
  "return_date": string (YYYY-MM-DD) or null,
  "days": integer or null,
  "budget_per_person": float or null,
  "interests": string or null,
  "transport_constraints": object or null,
  "hotel_constraints": object or null


User query:
{{text}}

Respond ONLY with valid JSON matching the model above. Do not include any explanation, markdown, or extra text.
"""

planner_prompt = PromptTemplate(
    input_variables=["text"],
    template=planner_prompt_template
)
planner_chain = planner_prompt | llm

In [9]:
# Define supervisor node function
def supervisor_node(state: PlannerState) -> PlannerState:
    goal = state.get("goal")
    if not goal:
        raise ValueError("Goal is required in the state.")

    result = supervisor_chain.invoke({"goal": goal})
    print(f"Supervisor raw output: {result.type}")
    try:
        
        return {
            "goal": goal,
            "intent": result.type,
            "final_plan": None
        }
    except ValidationError as e:
        print(f"Validation error parsing intention: {e}")
        return {
            "goal": goal,
            "intent": "unrelated",
            "final_plan": None
        }

In [10]:
# Define supervisor gate
def supervisor_gate(state: PlannerState) -> Literal["plan", "unrelated"]:
    if state.get("intent") in ["travel_plan", "hotel_only", "transport_only"]:
        return "plan"
    return "unrelated"

In [11]:
def planner_node(state: PlannerState) :
    goal = state.get("goal")
    if not goal:
        raise ValueError("Goal is required in the state.")

    try:
        constraints = planner_chain.invoke({"text": goal})
        print(constraints.content)
        # Convert to Constraints model
        constraints_model = constraints_parser.parse(constraints.content)
    except Exception as e:
        print("Extractor parse error:", e)
        constraints_model = Constraints()
    # return {
    #     "goal": state["goal"],
    #     "intent": state["intent"],
    #     "constraints": constraints_model,
    #     "final_plan": {"message": "Planning logic not yet implemented."}
    # }
    # state["constraints"] = constraints
    if state.get("intent") == "hotel_only":
        print("Going to hotel node")
        return Command(goto="hotel", update={"constraints": constraints_model})
    elif state.get("intent") == "transport_only":
        return Command(goto="transport", update={"constraints": constraints_model})
    else:
        return Command(goto=["transport", "hotel", "itinerary"], update={"constraints": constraints_model})
    # return state

In [12]:
# Define end unrelated node
def end_unrelated_node(state: PlannerState) -> PlannerState:
    return {
        "goal": state["goal"],
        "intent": state["intent"],
        "final_plan": {"message": "The request was unrelated to travel planning."}
    }

In [ ]:
# Define the prompt templates and chains for transport agent, hotel_agent, and itinerary_agent

transport_prompt_template = (
    "You are an expert travel assistant. Extract and return a JSON array of TransportOption objects (at least one) "
    "from the following transport search results. Respond ONLY with valid JSON matching the model below. "
    "Do not include any explanation, markdown, or extra text.\n\n"
    "Raw transport search results:\n"
    "{search_results}\n\n"
    "TransportOption model:\n"
    "  \"provider\": string,\n"
    "  \"mode\": string,\n"
    "  \"depart\": string,\n"
    "  \"arrive\": string,\n"
    "  \"duration_min\": string or null,\n"
    "  \"price_per_person\": string or null,\n"
    "  \"notes\": string or null"
)
transport_prompt = PromptTemplate(
    input_variables=["search_results"],
    template=transport_prompt_template
)
transport_chain = transport_prompt | llm

hotel_prompt_template = (
    "You are an expert travel assistant. Extract and return a JSON array of HotelOption objects (at least one) "
    "from the following hotel search results. Respond ONLY with valid JSON matching the model below. "
    "Do not include any explanation, markdown, or extra text.\n\n"
    "Raw hotel search results:\n"
    "{search_results}\n\n"
    "HotelOption model:\n"
    "  \"name\": string,\n"
    "  \"rating\": float or null,\n"
    "  \"price_per_night\": float or 0,\n"
    "  \"distance_from_center_km\": float or null,\n"
    "  \"amenities\": array of strings,\n"
    "  \"notes\": string or null"
)
hotel_prompt = PromptTemplate(
    input_variables=["search_results"],
    template=hotel_prompt_template
)
hotel_chain = hotel_prompt | llm

itinerary_prompt_template = (
    "You are an expert travel assistant that creates a day-wise itinerary based on the following search results and user constraints. "
    "Your response must be a JSON array of ItineraryItem objects. "
    "Do not include any explanation, markdown, or extra text.\n\n"
    "Raw itinerary search results:\n"
    "{search_results}\n\n"
    "User constraints:\n"
    "{constraints}\n\n"
    "ItineraryItem model:\n"
    "  \"day\": integer,\n"
    "  \"morning\": string,\n"
    "  \"afternoon\": string,\n"
    "  \"evening\": string,\n"
    "  \"notes\": string or null (optional)\n"
    "Create a detailed day-wise itinerary that matches the constraints."
)
itinerary_prompt = PromptTemplate(
    input_variables=["search_results", "constraints"],
    template=itinerary_prompt_template
)
itinerary_chain = itinerary_prompt | llm

In [81]:
#  Define the transport node
def transport_node(state: PlannerState) -> PlannerState:
    print("In transport node")
    constraints = state.get("constraints")
    if not constraints:
        raise ValueError("Constraints are required in the state for transport planning.")

    constraints_dict = constraints.model_dump() if hasattr(constraints, "dict") else constraints
    origin = constraints_dict.get("origin") or "any city in India"
    city = constraints_dict.get("city") or "any city in India"
    date = constraints_dict.get("date") or ""
    days = constraints_dict.get("days") or 1
    budget = constraints_dict.get("budget_per_person") or "any"
    interests = constraints_dict.get("interests") or "none"
    query = (
        f"Transport options from {origin} to {city} on {date} for {days} day(s), "
        f"budget {budget}, interests: {interests}. "
        "Return provider, mode, depart, arrive, duration in minutes, price per person, and notes."
    )
    print("Transport search query:", query)

    search_result = tavily_tool.invoke({"query": query})
    print("Raw transport search result:", search_result)

    llm_input = {"search_results": search_result}

    try:
        transport_options = transport_chain.invoke(llm_input)
        print("LLM transport options raw output:", transport_options)
        transport_options = getattr(transport_options, "content", transport_options)
        print("LLM transport options content:", transport_options)
        transport_options_list = json.loads(transport_options)
        transport_options = [TransportOption(**item) for item in transport_options_list]
    except Exception as e:
        print("Transport parse error:", e)
        transport_options = []

    return {
        "transport_options": transport_options
    }

In [86]:
# Define the hotel node
def hotel_node(state: PlannerState) -> PlannerState:
    print("In hotel node")
    constraints = state.get("constraints")
    if not constraints:
        raise ValueError("Constraints are required in the state for hotel planning.")

    constraints_dict = constraints.model_dump() if hasattr(constraints, "dict") else constraints
    city = constraints_dict.get("city") or "any city in India"
    date = constraints_dict.get("date") or ""
    days = constraints_dict.get("days") or 1
    budget = constraints_dict.get("budget_per_person") or "any"
    interests = constraints_dict.get("interests") or "none"
    query = (
        f"Hotels in {city} check-in on {date} for {days} night(s), "
        f"budget {budget}, interests: {interests}. "
        "Return hotel name, rating, price per night, distance from center, and amenities."
    )
    print("Hotel search query:", query)

    search_result = tavily_tool.invoke({"query": query})
    print("Raw search result:", search_result)

    llm_input = {"search_results": search_result}

    try:
        hotel_options = hotel_chain.invoke(llm_input)
        print("LLM hotel options raw output:", hotel_options)
        hotel_options = getattr(hotel_options, "content", hotel_options)
        print("LLM hotel options content:", hotel_options)
        
        hotel_options_list = json.loads(hotel_options)
        hotel_options = [HotelOption(**item) for item in hotel_options_list]
    except Exception as e:
        print("Hotel parse error:", e)
        hotel_options = []

    return {
        "hotel_options": hotel_options
    }

In [87]:
# Define itinerary node
def itinerary_node(state: PlannerState) -> PlannerState:
    constraints = state.get("constraints")
    if not constraints:
        raise ValueError("Constraints are required in the state for itinerary planning.")

    # Prepare a query for Tavily based on constraints
    constraints_dict = constraints.model_dump() if hasattr(constraints, "dict") else constraints
    city = constraints_dict.get("city") or "any city in India"
    days = constraints_dict.get("days") or 1
    interests = constraints_dict.get("interests") or "none"
    query = (
        f"Tourist attractions and activities in {city} for a {days}-day trip. "
        f"Interests: {interests}. Suggest morning, afternoon, and evening activities for each day."
    )
    print("Itinerary search query:", query)

    # Call Tavily tool to get search results
    search_result = tavily_tool.invoke({"query": query})
    print("Raw itinerary search result:", search_result)

    # Prepare input for LLM
    constraints_str = (
        json.dumps(constraints_dict, indent=2)
        if isinstance(constraints_dict, dict)
        else str(constraints_dict)
    )
    llm_input = {
        "search_results": search_result,
        "constraints": constraints_str
    }

    try:
        itinerary = itinerary_chain.invoke(llm_input)
        print(f"LLM itinerary raw output: {itinerary}")
        itinerary_content = getattr(itinerary, "content", itinerary)
        print(f"LLM itinerary content: {itinerary_content}")
        itinerary_list = json.loads(itinerary_content)
        print(f"Parsed itinerary_list: {itinerary_list}")
        if not isinstance(itinerary_list, list):
            raise ValueError("LLM did not return a JSON array.")
        itinerary = [ItineraryItem(**item) for item in itinerary_list]
    except Exception as e:
        print("Itinerary parse error:", e)
        itinerary = []

    return {
        "itinerary": itinerary
    }

In [17]:
# Define prompt templates and chains for final plan summarization
final_plan_prompt_template = """
You are an expert travel assistant that summarizes the final travel plan based on user constraints, transport options, hotel options, and itinerary.
Your response must be a summarirze text containing the final travel plan details with the following sections:
1. Overview: A brief summary of the trip including origin, destination, dates, duration, budget, and interests.
2. Transport Options: A summary of the selected transport options including provider, mode, depart, arrive, duration, price, and notes.
3. Hotel Options: A summary of the selected hotel options including name, rating, price, distance from center, amenities, and notes.
4. Itinerary: A day-wise breakdown of the itinerary including morning, afternoon, evening activities and notes.
User constraints:
{constraints}
Transport options:
{transport_options}
Hotel options:
{hotel_options}
Itinerary:
{itinerary}
Summarize the final travel plan in a clear and concise manner.
"""
final_plan_prompt = PromptTemplate(
    input_variables=["constraints", "transport_options", "hotel_options", "itinerary"],
    template=final_plan_prompt_template
)
final_plan_chain = final_plan_prompt | llm

In [18]:
# Define the summarizer node
def summarizer_node(state: PlannerState) -> PlannerState:
    constraints = state.get("constraints")
    transport_options = state.get("transport_options", [])
    hotel_options = state.get("hotel_options", [])
    itinerary = state.get("itinerary", [])
    
    if not constraints:
        raise ValueError("Constraints are required in the state for final plan summarization.")
    
    try:
        final_plan_summary = final_plan_chain.invoke({
            "constraints": constraints,
            "transport_options": transport_options,
            "hotel_options": hotel_options,
            "itinerary": itinerary
        })
        print(f"Final plan summary raw output: {final_plan_summary}")
    except Exception as e:
        print("Final plan summarization parse error:", e)
        final_plan_summary = {"message": "Could not generate final plan summary."}
    
    return {
        "goal": state["goal"],
        "intent": state["intent"],
        "constraints": constraints,
        "transport_options": transport_options,
        "hotel_options": hotel_options,
        "itinerary": itinerary,
        "final_plan": final_plan_summary
    }

In [88]:
# Build and complile the state graph
builder = StateGraph(PlannerState)

# Add nodes
builder.add_node("supervisor", supervisor_node)
builder.add_node("planner", planner_node)
builder.add_node("end_unrelated", end_unrelated_node)
builder.add_node("transport", transport_node)
builder.add_node("hotel", hotel_node)
builder.add_node("itinerary", itinerary_node)
builder.add_node("final_summarizer", summarizer_node)

# Add edges
builder.add_edge(START, "supervisor")
builder.add_conditional_edges("supervisor", supervisor_gate, {
    "plan": "planner",
    "unrelated": "end_unrelated"
})
# builder.add_edge("planner", END)
builder.add_edge("transport", "final_summarizer")
builder.add_edge("hotel", "final_summarizer")
builder.add_edge("itinerary", "final_summarizer")
builder.add_edge("final_summarizer", END)
builder.add_edge("end_unrelated", END)

# Compile the graph
graph = builder.compile()

In [89]:
# Test the graph with a travel planning goal
input_text = "Plan a 2-day trip to Varanasi for two adults from Asansol"
initial_state = {
    "goal": input_text,
    "intent": None,
    "final_plan": None
}
result = graph.invoke(initial_state)
print(result)

Supervisor raw output: travel_plan
{
  "origin": "Asansol",
  "city": "Varanasi",
  "date": null,
  "return_date": null,
  "days": 2,
  "budget_per_person": null,
  "interests": null,
  "transport_constraints": null,
  "hotel_constraints": null
}
In hotel node
Hotel search query: Hotels in Varanasi check-in on  for 2 night(s), budget any, interests: none. Return hotel name, rating, price per night, distance from center, and amenities.
Itinerary search query: Tourist attractions and activities in Varanasi for a 2-day trip. Interests: none. Suggest morning, afternoon, and evening activities for each day.
In transport node
Transport search query: Transport options from Asansol to Varanasi on  for 2 day(s), budget any, interests: none. Return provider, mode, depart, arrive, duration in minutes, price per person, and notes.
Raw transport search result: {'query': 'Transport options from Asansol to Varanasi on  for 2 day(s), budget any, interests: none. Return provider, mode, depart, arrive, 

In [83]:
# Test the graph for transport only
input_text= "I want to go to Mumbai from Asansol on 2nd October, 2025. What are the convenient transport options?"
result = graph.invoke({"goal": input_text})
print(result)

Supervisor raw output: transport_only
{
  "origin": "Asansol",
  "city": "Mumbai",
  "date": "2025-10-02",
  "return_date": null,
  "days": null,
  "budget_per_person": null,
  "interests": null,
  "transport_constraints": null,
  "hotel_constraints": null
}
In transport node
Transport search query: Transport options from Asansol to Mumbai on 2025-10-02 for 1 day(s), budget any, interests: none. Return provider, mode, depart, arrive, duration in minutes, price per person, and notes.
Raw transport search result: {'query': 'Transport options from Asansol to Mumbai on 2025-10-02 for 1 day(s), budget any, interests: none. Return provider, mode, depart, arrive, duration in minutes, price per person, and notes.', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.rome2rio.com/s/%C4%80sansol/Mumbai', 'title': 'Āsansol to Mumbai - 5 ways to travel via train, plane, and car', 'content': 'The cheapest way to get from Āsansol to Mumbai costs only $37, and 

In [74]:
# Test the graph for hotel only
input_text= "I want to go to Mumbai on 2nd October, 2025. Where should I stay?"
result = graph.invoke({"goal": input_text})
print(result)

Supervisor raw output: hotel_only
{
  "origin": null,
  "city": "Mumbai",
  "date": "2025-10-02",
  "return_date": null,
  "days": null,
  "budget_per_person": null,
  "interests": null,
  "transport_constraints": null,
  "hotel_constraints": null
}
Going to hotel node
In hotel node
Hotel search query: Hotels in Mumbai check-in on 2025-10-02 for 1 night(s), budget any, interests: none. Return hotel name, rating, price per night, distance from center, and amenities.
Raw search result: {'query': 'Hotels in Mumbai check-in on 2025-10-02 for 1 night(s), budget any, interests: none. Return hotel name, rating, price per night, distance from center, and amenities.', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.tripadvisor.com/LastMinute-g304554-Mumbai_Maharashtra-Hotels.html', 'title': 'The Best Last Minute Hotels in Mumbai 2025 - Tripadvisor', 'content': 'The Best Last Minute Hotels in Mumbai 2025 - Tripadvisor Mumbai Hotels Last Minute Hotels i

In [172]:
# Test the graph for unrelated
input_text= "What is the capital of Maharashtra?"
result = graph.invoke({"goal": input_text})
print(result)

Supervisor raw output: unrelated
{'goal': 'What is the capital of Maharashtra?', 'intent': 'unrelated', 'final_plan': {'message': 'The request was unrelated to travel planning.'}}
